In [8]:
import pandas as pd
import math
import csv
from pathlib import Path

In [5]:
def marks_to_percentage(score:int,total:int) -> float:
    return score*100/total

def percentage_to_marks(percentage:float,total:int) -> int:
    return floor(percentage*total)

In [6]:
a = -0.1035
def percentage_to_percentile(percentage:float) -> float:
    percentile = 100*(1-exp(a*percentage))
    return percentile

def percentile_to_percentage(percentile:float) -> float:
    percentage = (1/a)*ln(1-percentile/100)
    return percentage


In [7]:
b = 1500000
def percentile_to_air(percentile:float) -> int:
    return b*(1-percentile/100)

def air_to_percentile(air:int) -> float:
    return 100*(1-AIR/b)

In [10]:
# ============================
# Coefficient Declarations
# ============================

# Function 1 coefficients (SC)
F1_C1_M = 0.0251
F1_C1_B = -19.5

F1_C2_M = 0.0276
F1_C2_B = -51.9

F1_C3_M = 0.0383
F1_C3_B = -373

F1_C4_M = 0.0429
F1_C4_B = -605

F1_C5_M = 0.0515
F1_C5_B = -1297

F1_C6_M = 0.0571
F1_C6_B = -1854

F1_C7_M = 0.0738
F1_C7_B = -4542

F1_C8_M = 0.0892
F1_C8_B = -9217

F1_C9_M = 0.106
F1_C9_B = -17937

F1_C10_M = 0.118
F1_C10_B = -30183


# Function 2 coefficients (OBC-NCL)
F2_C1_M = 0.232
F2_C1_B = -131

F2_C2_M = 0.313
F2_C2_B = -1180

F2_C3_M = 0.351
F2_C3_B = -2833

F2_C4_M = 0.389
F2_C4_B = -7865


# Function 3 coefficients (EWS)
F3_C1_M = 0.129
F3_C1_B = -77.2

F3_C2_M = 0.145
F3_C2_B = -100

F3_C3_M = 0.118
F3_C3_B = 7517

F3_C4_M = 0.098
F3_C4_B = 19862

F3_C5_M = 0.0788
F3_C5_B = 39286


# Function 4 coefficients (ST)
F4_C1_M = 0.00725
F4_C1_B = -32.2

F4_C2_M = 0.0122
F4_C2_B = -326

F4_C3_M = 0.0165
F4_C3_B = -930

F4_C4_M_LINEAR = 0.0136
F4_C4_M_QUAD = 1.76E-8
F4_C4_B = -1146

F4_C5_M = -11081
F4_C5_B = 0.0396


# ============================
# Core Functions
# ============================

def air_to_cat(category: str, rank: float) -> float:
    """
    Route to appropriate function based on category.
    """
    if category == "SC":
        raw = calculate_function1(rank)
    elif category == "OBC-NCL":
        raw = calculate_function2(rank)
    elif category == "EWS":
        raw = calculate_function3(rank)
    elif category == "OPEN":
        raw = rank
    elif category == "ST":
        raw = calculate_function4(rank)
    else:
        raise ValueError("Invalid category")
    
    rounded = int(round(raw))
    
    if rounded <= 0:
        return 10
    return rounded


def calculate_function1(value: float) -> float:
    if value < 10000:
        return F1_C1_M * value + F1_C1_B
    elif 10000 <= value <= 30000:
        return F1_C2_M * value + F1_C2_B
    elif 30000 <= value <= 50000:
        return F1_C3_M * value + F1_C3_B
    elif 50000 <= value <= 75000:
        return F1_C4_M * value + F1_C4_B
    elif 75000 <= value <= 100000:
        return F1_C5_M * value + F1_C5_B
    elif 100000 <= value <= 150000:
        return F1_C6_M * value + F1_C6_B
    elif 150000 <= value <= 300000:
        return F1_C7_M * value + F1_C7_B
    elif 300000 <= value <= 500000:
        return F1_C8_M * value + F1_C8_B
    elif 500000 <= value <= 1000000:
        return F1_C9_M * value + F1_C9_B
    elif value > 1000000:
        return F1_C10_M * value + F1_C10_B
    return 0.0  # fallback


def calculate_function2(value: float) -> float:
    if value < 10000:
        return F2_C1_M * value + F2_C1_B
    elif 10000 <= value <= 50000:
        return F2_C2_M * value + F2_C2_B
    elif 50000 <= value <= 100000:
        return F2_C3_M * value + F2_C3_B
    elif value > 100000:
        return F2_C4_M * value + F2_C4_B
    return 0.0  # fallback


def calculate_function3(value: float) -> float:
    if value <= 10000:
        return F3_C1_M * value + F3_C1_B
    elif 10000 < value <= 300000:
        return F3_C2_M * value + F3_C2_B
    elif 300000 < value <= 600000:
        return F3_C3_M * value + F3_C3_B
    elif 600000 < value <= 1000000:
        return F3_C4_M * value + F3_C4_B
    elif value > 1000000:
        return F3_C5_M * value + F3_C5_B
    return 0.0  # fallback


def calculate_function4(value: float) -> float:
    if value <= 50000:
        return F4_C1_M * value + F4_C1_B
    elif 50000 < value <= 150000:
        return F4_C2_M * value + F4_C2_B
    elif 150000 < value <= 200000:
        return F4_C3_M * value + F4_C3_B
    elif 200000 < value <= 750000:
        return F4_C4_M_LINEAR * (value ** 2) + F4_C4_M_QUAD * VALUE + F4_C4_B
    elif value > 750000:
        return F4_C5_CONST + F4_C5_M * value
    return 0.0  # fallback

In [11]:
# ============================
# Coefficient Declarations
# ============================

# Inverse Function 1 coefficients (OBC-NCL)
INV1_C1_M = 0.232
INV1_C1_B = 131       # (y + 131) / 0.232

INV1_C2_M = 0.313
INV1_C2_B = 1180      # (y + 1180) / 0.313

INV1_C3_M = 0.351
INV1_C3_B = 2833      # (y + 2833) / 0.351

INV1_C4_M = 0.389
INV1_C4_B = 7865      # (y + 7865) / 0.389


# Inverse Function 2 coefficients (EWS)
INV2_C1_M = 0.129
INV2_C1_B = 77.2      # (y + 77.2) / 0.129

INV2_C2_M = 0.145
INV2_C2_B = 100       # (y + 100) / 0.145

INV2_C3_M = 0.118
INV2_C3_B = -7517     # (y - 7517) / 0.118

INV2_C4_M = 0.098
INV2_C4_B = -19862    # (y - 19862) / 0.098

INV2_C5_M = 0.0788
INV2_C5_B = -39286    # (y - 39286) / 0.0788


# Inverse Function 3 coefficients (ST)
INV3_C1_M = 0.0267
INV3_C1_B = 33.5      # (y + 33.5) / 0.0267

INV3_C2_M = 0.0404
INV3_C2_B = 453       # (y + 453) / 0.0404

INV3_C3_M = 0.0561
INV3_C3_B = 1724      # (y + 1724) / 0.0561

INV3_C4_M = 0.0739
INV3_C4_B = 4548      # (y + 4548) / 0.0739

# Quadratic inverse coefficients for ST
INV3_Q_A = 1.73e-8
INV3_Q_B = 0.0783
INV3_Q_C_CONST = -7190  # c = -7190 - y  =>  C(y) = INV3_Q_C_CONST - y


# Inverse Function 4 coefficients (SC)
INV4_C1_M = 0.0251
INV4_C1_B = 19.5      # (y + 19.5) / 0.0251

INV4_C2_M = 0.0276
INV4_C2_B = 51.9      # (y + 51.9) / 0.0276

INV4_C3_M = 0.0383
INV4_C3_B = 373       # (y + 373) / 0.0383

INV4_C4_M = 0.0429
INV4_C4_B = 605       # (y + 605) / 0.0429

INV4_C5_M = 0.0515
INV4_C5_B = 1297      # (y + 1297) / 0.0515

INV4_C6_M = 0.0571
INV4_C6_B = 1854      # (y + 1854) / 0.0571

INV4_C7_M = 0.0738
INV4_C7_B = 4542      # (y + 4542) / 0.0738

INV4_C8_M = 0.0892
INV4_C8_B = 9217      # (y + 9217) / 0.0892

INV4_C9_M = 0.106
INV4_C9_B = 17937     # (y + 17937) / 0.106

INV4_C10_M = 0.118
INV4_C10_B = 30183    # (y + 30183) / 0.118


# ============================
# Core Functions
# ============================

def cat_to_air(category: str, rank: float) -> float:
    """
    Route to appropriate inverse function based on category.
    """
    if category == "SC":
        raw = inverse_function4(rank)
    elif category == "OBC-NCL":
        raw = inverse_function1(rank)
    elif category == "EWS":
        raw = inverse_function2(rank)
    elif category == "OPEN":
        raw = rank
    elif category == "ST":
        raw = inverse_function3(rank)
    else:
        raise ValueError("Invalid category")
    rounded = int(round(raw))
    if rounded < 0:
        return 10
    return rounded


def inverse_function1(y: float) -> float:
    if y < 2189:
        return (y + INV1_C1_B) / INV1_C1_M
    elif 2189 <= y <= 14470:
        return (y + INV1_C2_B) / INV1_C2_M
    elif 14470 <= y <= 32267:
        return (y + INV1_C3_B) / INV1_C3_M
    elif y > 32267:
        return (y + INV1_C4_B) / INV1_C4_M
    return 0.0  # optional fallback, JS version just returns undefined


def inverse_function2(y: float) -> float:
    if y <= 1212.8:
        return (y + INV2_C1_B) / INV2_C1_M
    elif 1212.8 < y <= 43400:
        return (y + INV2_C2_B) / INV2_C2_M
    elif 43400 < y <= 78317:
        return (y + INV2_C3_B) / INV2_C3_M
    elif 78317 < y <= 117862:
        return (y + INV2_C4_B) / INV2_C4_M
    elif y > 117862:
        return (y + INV2_C5_B) / INV2_C5_M
    return 0.0  # optional fallback


def inverse_function3(y: float) -> int:
    """
    Inverse of calculate_function4(value), with post-processing:
      - Round the resulting x to the nearest integer.
      - If the rounded value is <= 0, return 10 instead.
    """
    # ----------------------------
    # Determine y-ranges of each segment
    # ----------------------------
    # Segment 1 range: x in [0, 50000]  (assuming domain starts at 0)
    y_s1_min = calculate_function4(0.0)          # -32.2
    y_s1_max = calculate_function4(50000.0)
    # Segment 2 range: (50000, 150000]
    y_s2_min = calculate_function4(50000.0)
    y_s2_max = calculate_function4(150000.0)
    # Segment 3 range: (150000, 200000]
    y_s3_min = calculate_function4(150000.0)
    y_s3_max = calculate_function4(200000.0)
    # Segment 4 range: (200000, 750000]
    y_s4_min = calculate_function4(200000.0)
    y_s4_max = calculate_function4(750000.0)
    # Segment 5 range: (750000, +inf)
    y_s5_min = calculate_function4(750000.0)
    # y_s5_max = +inf (unbounded, linear increasing)
    # ----------------------------
    # Invert per segment by y-range
    # ----------------------------
    # Segment 1: linear
    if y_s1_min <= y <= y_s1_max:
        x = (y - C4_S1_B) / C4_S1_A
    # Segment 2: linear
    elif y_s2_min < y <= y_s2_max:
        x = (y - C4_S2_B) / C4_S2_A
    # Segment 3: linear
    elif y_s3_min < y <= y_s3_max:
        x = (y - C4_S3_B) / C4_S3_A
    # Segment 4: quadratic
    elif y_s4_min < y <= y_s4_max:
        # Solve: C4_S4_A * x^2 + C4_S4_B * x + (C4_S4_C - y) = 0
        a = C4_S4_A
        b = C4_S4_B
        c = C4_S4_C - y
        disc = b * b - 4.0 * a * c
        if disc < 0:
            raise ValueError("No real solution for quadratic inverse in segment 4")
        sqrt_disc = math.sqrt(disc)
        # Two roots
        x1 = (-b + sqrt_disc) / (2.0 * a)
        x2 = (-b - sqrt_disc) / (2.0 * a)
        # Select the root that lies in (200000, 750000]
        x = None
        for candidate in (x1, x2):
            if 200000.0 < candidate <= 750000.0:
                x = candidate
                break
        if x is None:
            raise ValueError("No valid root in the domain (200000, 750000] for segment 4")
    # Segment 5: linear
    elif y > y_s5_min:
        x = (y - C4_S5_B) / C4_S5_A
        if x <= 750000.0:
            raise ValueError("Inverse in segment 5 produced x not > 750000")
    else:
        # If y doesn't fall into any range:
        raise ValueError("Input y is outside the range of calculate_function4")


def inverse_function4(y: float) -> float:
    if y < 231.6:
        return (y + INV4_C1_B) / INV4_C1_M
    elif 231.6 <= y <= 776.1:
        return (y + INV4_C2_B) / INV4_C2_M
    elif 776.1 <= y <= 1542:
        return (y + INV4_C3_B) / INV4_C3_M
    elif 1542 <= y <= 2612.5:
        return (y + INV4_C4_B) / INV4_C4_M
    elif 2612.5 <= y <= 3853:
        return (y + INV4_C5_B) / INV4_C5_M
    elif 3853 <= y <= 6711:
        return (y + INV4_C6_B) / INV4_C6_M
    elif 6711 <= y <= 17598:
        return (y + INV4_C7_B) / INV4_C7_M
    elif 17598 <= y <= 35483:
        return (y + INV4_C8_B) / INV4_C8_M
    elif 35483 <= y <= 88063:
        return (y + INV4_C9_B) / INV4_C9_M
    elif y > 88063:
        return (y + INV4_C10_B) / INV4_C10_M
    return 0.0  # optional fallback

In [12]:
def rank_predictor(cat:str, marks:int) -> int:
    percentage = marks_to_percentage(marks)
    percentile = percentage_to_percentile(percentage)
    air = percentile_to_air(percentile)
    cat_rank = air_to_cat(cat,air)
    return cat_rank

def marks_predictor(cat:str, rank:int) -> int:
    air = cat_to_air(cat, rank)
    percentile = air_to_percentile(air)
    percentage = percentile_to_percentage(percentile)
    marks = percentage_to_marks(percentage)
    return marks